In [1]:
import pandas as pd

In [2]:
#Leo el csv de eventos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

## Generacion de features

In [3]:
features_df = pd.DataFrame()

In [4]:
features_df['person'] = events_df.person.unique()

In [5]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


## Cantidad de eventos total

In [6]:
df = events_df.groupby('person').event.count()

In [7]:
features_df = features_df.join(df, on='person')

In [8]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

In [9]:
features_df.head(3)

,person,cant_eventos_totales
0,4886f805,9
1,ad93850f,65
2,0297fc1e,567


## Cantidad de dias en los que entra

In [ ]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['dia'] = events_df['timestamp'].dt.day
events_df['hora'] = events_df['timestamp'].dt.hour
events_df['weekday'] = events_df['timestamp'].dt.day_name()

In [ ]:
events_df.head()

In [ ]:
ct = pd.crosstab(events_df.person, events_df.weekday)
ct.head()

In [ ]:
features_df = features_df.join(ct, on='person')

## Quincenas

In [ ]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['mes'] = events_df['timestamp'].dt.month
events_df['dia'] = events_df['timestamp'].dt.day

In [ ]:
events_df['q1'] = ((events_df['mes']==1) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q2'] = ((events_df['mes']==1) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q3'] = ((events_df['mes']==2) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q4'] = ((events_df['mes']==2) & (events_df['dia']>15) & (events_df['dia']<29))
events_df['q5'] = ((events_df['mes']==3) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q6'] = ((events_df['mes']==3) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q7'] = ((events_df['mes']==4) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q8'] = ((events_df['mes']==4) & (events_df['dia']>15) & (events_df['dia']<31))
events_df['q9'] = ((events_df['mes']==5) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q10'] = ((events_df['mes']==5) & (events_df['dia']>15) & (events_df['dia']<32))

In [ ]:
df = events_df[['person', 'q1','q2','q3','q4','q5','q6','q7','q8','q9','q10']].groupby('person').agg(lambda x: x.any())
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

## Mean encoding channel 

In [10]:
#Mean encoding

labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [11]:
#Separo en train y to_predict
mean_encoding = events_df.merge(labels_df, on='person', how='right')
mean_encoding.shape

(1171886, 24)

In [12]:
me = mean_encoding.groupby(['channel']).label.value_counts(normalize=True).unstack().drop(columns=[0]).rename(columns=({1:'mean_enc_channel'}))
me

label,mean_enc_channel
channel,
Direct,0.112726
Email,0.107784
Organic,0.092337
Paid,0.066072
Referral,0.101998
Social,0.076923
Unknown,NaN


In [13]:
events_df = events_df.join(me, on='channel')

In [ ]:
#todo['me_channel'] = events_df.mean_enc_channel

In [ ]:
events_df.channel.value_counts()

In [ ]:
events_df.head(3)

In [ ]:
events_df.channel.value_counts()

In [ ]:
events_df.mean_enc_channel.value_counts()

In [14]:
df = events_df.groupby(['person']).mean_enc_channel.mean()
df.head(3)

person
0008ed71    0.101998
00091926    0.101433
00091a7a    0.066072
Name: mean_enc_channel, dtype: float64

In [15]:
features_df = features_df.join(df, on='person')

In [16]:
features_df.head()

,person,cant_eventos_totales,mean_enc_channel
0,4886f805,9,0.092337
1,ad93850f,65,0.071325
2,0297fc1e,567,0.100279
3,2d681dd8,26,0.079205
4,cccea85e,836,0.086540


## Mean encoding storage

In [17]:
me = mean_encoding.groupby(['storage']).label.value_counts(normalize=True).unstack().drop(columns=[0]).rename(columns=({1:'mean_enc_storage'}))
me.head()

label,mean_enc_storage
storage,
128GB,0.078651
16GB,0.082392
256GB,0.064498
32GB,0.085803
4GB,0.084156


In [ ]:
events_df.storage.value_counts()

In [18]:
events_df = events_df.join(me, on='storage')

In [ ]:
#todo['me_storage'] = events_df.mean_enc_storage

In [ ]:
events_df.mean_enc_storage.value_counts()

In [19]:
df = events_df.groupby(['person']).mean_enc_storage.mean()
df.head(3)

person
0008ed71    0.083973
00091926    0.082193
00091a7a    0.080452
Name: mean_enc_storage, dtype: float64

In [20]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.head()

## Mean encoding eventos

In [21]:
me = mean_encoding.groupby(['event']).label.value_counts(normalize=True).unstack().drop(columns=[0]).rename(columns=({1:'mean_enc_event'}))
me.head()

label,mean_enc_event
event,
ad campaign hit,0.064665
brand listing,0.105026
checkout,0.055226
conversion,0.122334
generic listing,0.095422


In [ ]:
events_df.event.value_counts()

In [22]:
events_df = events_df.join(me, on='event')

In [ ]:
#todo['me_event'] = events_df.mean_enc_event

In [ ]:
events_df.mean_enc_event.value_counts()

In [23]:
df = events_df.groupby(['person']).mean_enc_event.mean()
df.head(3)

person
0008ed71    0.072605
00091926    0.085856
00091a7a    0.093343
Name: mean_enc_event, dtype: float64

In [24]:
features_df = features_df.join(df, on='person')

In [25]:
features_df.head()

,person,cant_eventos_totales,mean_enc_channel,mean_enc_storage,mean_enc_event
0,4886f805,9,0.092337,0.085803,0.081804
1,ad93850f,65,0.071325,0.083315,0.086517
2,0297fc1e,567,0.100279,0.080981,0.084847
3,2d681dd8,26,0.079205,0.082251,0.086867
4,cccea85e,836,0.086540,0.085786,0.085314


## Mean encoding condition

In [29]:
me = mean_encoding.groupby(['condition']).label.value_counts(normalize=True).unstack().drop(columns=[0]).rename(columns=({1:'mean_enc_cond'}))
me.head()

label,mean_enc_cond
condition,
Bom,0.084797
Bom - Sem Touch ID,0.066789
Excelente,0.079714
Muito Bom,0.089109
Novo,0.113676


In [30]:
events_df = events_df.join(me, on='condition')

In [33]:
df = events_df.groupby(['person']).mean_enc_cond.mean()
df.head(3)

person
0008ed71    0.086235
00091926    0.085286
00091a7a    0.081409
Name: mean_enc_cond, dtype: float64

In [34]:
features_df = features_df.join(df, on='person')

In [35]:
#Separo en train y to_predict
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 7)

In [36]:
#Separo en train y to_predict
#train_df = todo.merge(labels_df, on='person', how='right')
#train_df.shape

In [37]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 6)

In [ ]:
#to_predict = todo[~todo.person.isin(labels_df.person)]
#to_predict.shape

In [38]:
# Labels: los valores a predecir
#labels = np.array(train_df.label)
labels = train_df.label

#remuevo el label del df
train_df = train_df.drop(columns=['label'])

In [39]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [40]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

## XGBoost

In [41]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [42]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [43]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5, 10],
        'colsample_bytree': [0.1, 0.2, 0.3, 0.5],
        'colsample_bylevel': [0.7, 0.8, 0.9],
        'max_depth': [3, 4, 5, 7, 8, 9, 11, 13, 15],
        'base_score': [0.1, 0.2, 0.3],
        'n_estimators': [35, 40, 50, 60, 70, 80, 90],
        'booster':['gbtree', 'dart']
        }

In [44]:
xg_gs = xgb.XGBRegressor(objective ='binary:logistic',
                        learning_rate = 0.2,
                        alpha = 20, eval_metric='auc')

In [45]:
folds = 10
param_comb = 7

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xg_gs, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, 
                                   cv=skf.split(train_df.drop(columns=['person']),labels), verbose=3, random_state=1001,return_train_score=True)

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(train_df.drop(columns=['person']), labels)
timer(start_time) # timing ends here for "start_time" variable

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   13.5s
[Parallel(n_jobs=4)]: Done  70 out of  70 | elapsed:   35.2s finished



 Time taken: 0 hours 0 minutes and 37.33 seconds.


In [46]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)

print('\n Best hyperparameters:')
print(random_search.best_params_)


 All results:
{'mean_fit_time': array([2.79040811, 1.51875899, 0.44955606, 0.22825587, 1.00926685,
       0.41571269, 5.50695386]), 'std_fit_time': array([0.03430354, 0.03045154, 0.01502139, 0.00357616, 0.0250462 ,
       0.00461418, 0.56503825]), 'mean_score_time': array([0.01264889, 0.00612288, 0.005477  , 0.00715578, 0.00560265,
       0.01022317, 0.01677432]), 'std_score_time': array([0.00105001, 0.0001864 , 0.00010442, 0.00030113, 0.00011292,
       0.00059526, 0.00174639]), 'param_n_estimators': masked_array(data=[70, 70, 35, 40, 70, 50, 70],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[1, 1, 10, 1, 5, 5, 1],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[5, 11, 4, 4, 5, 7, 15],
             mask=[False, False, False, False, False, False, False],
       

In [47]:
preds_grid_search = random_search.predict(X_test)

In [48]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds_grid_search)

0.8640194352875586

In [49]:
preds_posta = random_search.predict(to_predict.drop(columns=['person']))

In [50]:
preds_posta

array([0.01244762, 0.20672776, 0.11380915, ..., 0.01994108, 0.01994108,
       0.01994108], dtype=float32)

In [51]:
to_publish = pd.DataFrame()

In [52]:
to_publish['person'] = to_predict.person
to_publish.shape

(19415, 1)

In [53]:
to_publish['label'] = preds_posta
to_publish.shape

(19415, 2)

In [54]:
to_publish.head()

,person,label
0,4886f805,0.012448
1,0297fc1e,0.206728
2,2d681dd8,0.113809
3,cccea85e,0.207102
4,4c8a8b93,0.080165


In [55]:
to_publish.loc[ to_publish.label > 1 ]

,person,label


In [56]:
to_publish.loc[ to_publish.label < 0 ]

,person,label


In [57]:
to_publish.to_csv('3_12_2.csv', index=False)